In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

sys.path.append("../")

import torch
import transformers
import baukit
from tqdm.auto import tqdm
import json
import os
from src import functional
import src.tokens as tokenization_utils
import numpy as np
import logging
from src import models

from src.utils import logging_utils
logger = logging.getLogger(__name__)
logging.basicConfig(
    level=logging.DEBUG,
    format=logging_utils.DEFAULT_FORMAT,
    datefmt=logging_utils.DEFAULT_DATEFMT,
    stream=sys.stdout,
)

torch.__version__, transformers.__version__, torch.version.cuda

('2.1.2+cu121', '4.39.0.dev0', '12.1')

In [3]:
from src.models import ModelandTokenizer

# MODEL_PATH = "state-spaces/mamba-2.8b-slimpj" # state-spaces/mamba-2.8b
MODEL_PATH = "EleutherAI/pythia-2.8b-deduped"

mt = ModelandTokenizer(
    model_path=MODEL_PATH, 
    torch_dtype=torch.float32
)

2024-03-19 11:52:17 urllib3.connectionpool DEBUG    Starting new HTTPS connection (1): huggingface.co:443


2024-03-19 11:52:17 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /EleutherAI/pythia-2.8b-deduped/resolve/main/config.json HTTP/1.1" 200 0
2024-03-19 11:52:18 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /EleutherAI/pythia-2.8b-deduped/resolve/main/generation_config.json HTTP/1.1" 404 0
2024-03-19 11:52:19 urllib3.connectionpool DEBUG    https://huggingface.co:443 "HEAD /EleutherAI/pythia-2.8b-deduped/resolve/main/tokenizer_config.json HTTP/1.1" 200 0


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


2024-03-19 11:52:19 src.models INFO     loaded model <EleutherAI/pythia-2.8b-deduped> | size: 10724.583 MB | dtype: torch.float32 | device: cuda


In [4]:
#####################################################
subject = "The Space Needle"
prompt_template = "{} is located in the city of"
# prompt_template = tokenization_utils.maybe_prefix_eos(
#     mt.tokenizer, prompt_template
# )
#####################################################

prompt = prompt_template.format(subject)
prompt

'The Space Needle is located in the city of'

In [5]:
from src.functional import predict_next_token

predict_next_token(
    mt,
    prompt=prompt,
    k=5,
)

[[PredictedToken(token=' Seattle', prob=0.9797233939170837),
  PredictedToken(token=' Portland', prob=0.0019473006250336766),
  PredictedToken(token=' Vancouver', prob=0.0017969132168218493),
  PredictedToken(token=' Iss', prob=0.0013203793205320835),
  PredictedToken(token='Se', prob=0.0013112127780914307)]]

In [6]:
# from src.data.dataclasses import MultiCounterFactDataset

# dataset = MultiCounterFactDataset("../data")

request = {
    "prompt": prompt_template,
    "subject": subject,
    "target_new": {"str": "Paris"},
}

generation_prompts = [
    f"{subject} is located in the city of",
    f"{subject}, which is in the city of",
    f"Which city is the {subject} in? It is in",
    f"{subject} is made of",
    f"{subject} is in"
]

In [7]:
mt.model

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(50304, 2560)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear(in_features=2560, out_features=7680, bias=True)
          (dense): Linear(in_features=2560, out_features=2560, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear(in_features=2560, out_features=10240, bias=True)
          (dense_4h_to_h): Linear(in_features=10240, out_features=2560, bias=True)


In [8]:
from src.rome.compute_v import compute_v, get_module_input_output_at_word

context_templates=[
    '{}', 
    'The first step to a new life is to. {}', 
    'Therefore, the best way to prevent this from. {}', 
    'Because the first time I saw the trailer. {}', 
    "I'm not sure if this is the. {}", 
    'You are here: Home / Archives for . {}', 
]
words= [subject] * len(context_templates)

l_input, l_output = get_module_input_output_at_word(
    mt, 
    layer = 15,
    context_template = request["prompt"],
    word = request["subject"],
    module_template=mt.layer_name_format + ".mlp.dense_4h_to_h",
    fact_token_strategy="subject_last"
)

2024-03-19 11:52:27 numexpr.utils INFO     Note: NumExpr detected 24 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2024-03-19 11:52:27 numexpr.utils INFO     NumExpr defaulting to 8 threads.
2024-03-19 11:52:27 datasets INFO     PyTorch version 2.1.2 available.
2024-03-19 11:52:27 matplotlib DEBUG    matplotlib data path: /home/local_arnab/miniconda3/envs/relations/lib/python3.10/site-packages/matplotlib/mpl-data
2024-03-19 11:52:27 matplotlib DEBUG    CONFIGDIR=/home/local_arnab/.config/matplotlib
2024-03-19 11:52:27 matplotlib DEBUG    interactive is False
2024-03-19 11:52:27 matplotlib DEBUG    platform is linux
2024-03-19 11:52:27 src.rome.repr_tools DEBUG    ==> [([3], 'le')]


In [9]:
# from src.rome_utils import nethook

# tokenized = mt.tokenizer(prompt, return_tensors="pt", padding=True, return_offsets_mapping=True).to(mt.device)
# offsets = tokenized.pop("offset_mapping")

# [(idx, mt.tokenizer.decode(t)) for idx, t in enumerate(tokenized.input_ids[0])]

In [10]:
# with nethook.Trace(
#     module = mt.model,
#     layer = mt.layer_name_format.format(15) + ".mixer",
#     retain_output = True,
#     retain_input = True,
# ) as tr:
#     output = mt(**tokenized)

In [11]:
request["prompt"]

'{} is located in the city of'

In [12]:
mt.mlp_module_name_format

'gpt_neox.layers.{}.mlp'

In [13]:
from src.rome.rome_hparams import ROMEHyperParams

hparams = ROMEHyperParams(
    layers = [15],
    fact_token="subject_last",
    v_num_grad_steps=20,
    v_lr=5e-1,
    v_loss_layer=models.determine_layers(mt)[-1],
    v_weight_decay=0.5,
    clamp_norm_factor=3,
    kl_factor=0.0625,
    mom2_adjustment=True,
    context_template_length_params=[[5, 10], [10, 10]],

    rewrite_module_tmp=mt.layer_name_format + ".mlp.dense_4h_to_h",
    layer_module_tmp=mt.layer_name_format,
    mlp_module_tmp=mt.mlp_module_name_format,
    attn_module_tmp=mt.attn_module_name_format,
    ln_f_module=models.determine_final_layer_norm_path(mt),
    lm_head_module=models.determine_lm_head_path(mt),
    
    mom2_dataset="wikipedia",
    mom2_n_samples=1000,
    mom2_dtype="float32",
)


# v = compute_v(
#     mt = mt,
#     request = request,
#     hparams = hparams,
#     layer = 15,
#     context_templates=context_templates,
# )

In [15]:
import json
print(json.dumps(hparams.__dict__, indent=2))
# hparams.__dict__

{
  "layers": [
    15
  ],
  "fact_token": "subject_last",
  "v_num_grad_steps": 20,
  "v_lr": 0.5,
  "v_loss_layer": 31,
  "v_weight_decay": 0.5,
  "clamp_norm_factor": 3,
  "kl_factor": 0.0625,
  "mom2_adjustment": true,
  "context_template_length_params": [
    [
      5,
      10
    ],
    [
      10,
      10
    ]
  ],
  "rewrite_module_tmp": "gpt_neox.layers.{}.mlp.dense_4h_to_h",
  "layer_module_tmp": "gpt_neox.layers.{}",
  "mlp_module_tmp": "gpt_neox.layers.{}.mlp",
  "attn_module_tmp": "gpt_neox.layers.{}.attention",
  "ln_f_module": "gpt_neox.final_layer_norm",
  "lm_head_module": "embed_out",
  "mom2_dataset": "wikipedia",
  "mom2_n_samples": 1000,
  "mom2_dtype": "float32",
  "mamba_block_non_ssm": false,
  "mamba_block_ssm": false
}


In [24]:
from src.rome.rome_main import get_context_templates

get_context_templates(
    mt = mt,
    length_params=[[5, 10], [10, 10]]
)

['{}',
 'Q: . {}',
 'Q: . {}',
 'Q: . {}',
 ' A new. {}',
 'Q: . {}',
 '\n-2*. {}',
 '\nThe following is. {}',
 'Q: . {}',
 '\nThe present invention. {}',
 '\nIn a wireless. {}',
 'Q: How to make a custom. {}',
 'Q: How do I add a. {}',
 'Q: How to get the name. {}',
 'Q: How do I get the. {}',
 'The present invention relates to a new and distinct. {}',
 'Q: How to create a new. {}',
 'Q: How to use multiple conditions. {}',
 'Q: Is it possible to use. {}',
 'A novel approach in the treatment of chronic rh. {}',
 'The invention relates to an apparatus and a process. {}']

In [25]:
from src.rome.compute_v import compute_v

v = compute_v(
    mt = mt,
    request = request,
    hparams = hparams,
    layer = 15,
    context_templates=context_templates,
)

2024-03-19 11:34:14 src.rome.compute_v INFO     Computing right vector (v)
2024-03-19 11:34:14 src.rome.compute_v DEBUG    Lookup index found: 3 | Sentence: The Space Needle is located in the city of | Token:le
2024-03-19 11:34:14 src.rome.compute_v DEBUG    Lookup indices: [3, 13, 13, 12, 12, 12, 3]
2024-03-19 11:34:14 src.rome.compute_v INFO     Rewrite layer is 15
2024-03-19 11:34:14 src.rome.compute_v INFO     Tying optimization objective to layer 31
right_vector_shape=2560 | left_vector_shape=10240
2024-03-19 11:34:14 src.rome.compute_v DEBUG    Optimizing delta of shape torch.Size([2560]) at layer 15


2024-03-19 11:34:14 src.rome.compute_v INFO     Recording initial value of v*
2024-03-19 11:34:14 src.rome.compute_v INFO     loss 21.072 = 21.072 + 0.0 + 0.0 avg prob of [Paris] 0.00000
2024-03-19 11:34:15 src.rome.compute_v INFO     loss 6.911 = 6.863 + 0.003 + 0.045 avg prob of [Paris] 0.00119
2024-03-19 11:34:15 src.rome.compute_v INFO     loss 3.608 = 3.53 + 0.008 + 0.071 avg prob of [Paris] 0.03046
2024-03-19 11:34:15 src.rome.compute_v INFO     loss 1.494 = 1.389 + 0.015 + 0.089 avg prob of [Paris] 0.25899
2024-03-19 11:34:15 src.rome.compute_v INFO     loss 0.185 = 0.081 + 0.015 + 0.089 avg prob of [Paris] 0.92271
2024-03-19 11:34:15 src.rome.compute_v INFO     loss 0.117 = 0.012 + 0.015 + 0.089 avg prob of [Paris] 0.98794
2024-03-19 11:34:15 src.rome.compute_v INFO     loss 0.11 = 0.006 + 0.015 + 0.089 avg prob of [Paris] 0.99448
2024-03-19 11:34:15 src.rome.compute_v INFO     loss 0.108 = 0.004 + 0.014 + 0.089 avg prob of [Paris] 0.99596
2024-03-19 11:34:15 src.rome.compute_v

In [26]:
from src.rome.rome_main import (
    apply_rome_to_model,
    restore_weights,
    save_weights,
)

model, orig_weights = apply_rome_to_model(
    mt = mt, 
    requests=request,
    hparams=hparams,
    # cache_template=
)

rome_weights = save_weights(model, list(orig_weights.keys()))

Executing ROME algorithm for the update: [The Space Needle is located in the city of] -> [ Paris]
Computing left vector (u)...
Selected u projection object The Space Needle
2024-03-19 11:34:18 src.rome.repr_tools DEBUG    ==> [([3], 'le'), ([6], 'le'), ([6], 'le'), ([6], 'le'), ([6], 'le'), ([6], 'le'), ([7], 'le'), ([8], 'le'), ([6], 'le'), ([8], 'le'), ([8], 'le'), ([11], 'le'), ([11], 'le'), ([11], 'le'), ([11], 'le'), ([13], 'le'), ([11], 'le'), ([11], 'le'), ([11], 'le'), ([13], 'le'), ([13], 'le')]


Retrieving inverse covariance statistics for eleutherai_pythia-2.8b-deduped @ gpt_neox.layers.15.mlp.dense_4h_to_h. The result will be cached to avoid repetitive computation.
2024-03-19 11:34:18 src.rome.layer_stats DEBUG    context length set to 2048 tokens.
2024-03-19 11:34:18 src.rome.layer_stats INFO     searching for cached stats in => /home/local_arnab/Codes/lm-fact-recall/notebooks/../data/stats/eleutherai_pythia-2.8b-deduped/wikipedia_stats/gpt_neox.layers.15.mlp.dense_4h_to_h_float32_mom2_1000.npz
2024-03-19 11:34:18 src.rome.layer_stats INFO     stats not found locally.
2024-03-19 11:34:18 src.rome.layer_stats INFO     Attempting to download eleutherai_pythia-2.8b-deduped/wikipedia_stats/gpt_neox.layers.15.mlp.dense_4h_to_h_float32_mom2_1000.npz from https://memit.baulab.info/data/stats/eleutherai_pythia-2.8b-deduped/wikipedia_stats/gpt_neox.layers.15.mlp.dense_4h_to_h_float32_mom2_1000.npz.
2024-03-19 11:34:18 src.rome.layer_stats ERROR    Unable to download due to HTTP Erro

Loading dataset shards:   0%|          | 0/41 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Left vector shape: torch.Size([10240])
2024-03-19 11:36:39 src.rome.compute_v INFO     Computing right vector (v)
2024-03-19 11:36:39 src.rome.compute_v DEBUG    Lookup index found: 3 | Sentence: The Space Needle is located in the city of | Token:le
2024-03-19 11:36:39 src.rome.compute_v DEBUG    Lookup indices: [3, 6, 6, 6, 6, 6, 7, 8, 6, 8, 8, 11, 11, 11, 11, 13, 11, 11, 11, 13, 13, 3]
2024-03-19 11:36:39 src.rome.compute_v INFO     Rewrite layer is 15
2024-03-19 11:36:39 src.rome.compute_v INFO     Tying optimization objective to layer 31
right_vector_shape=2560 | left_vector_shape=10240
2024-03-19 11:36:39 src.rome.compute_v DEBUG    Optimizing delta of shape torch.Size([2560]) at layer 15
2024-03-19 11:36:40 src.rome.compute_v INFO     Recording initial value of v*
2024-03-19 11:36:40 src.rome.compute_v INFO     loss 15.347 = 15.347 + 0.0 + 0.0 avg prob of [ Paris] 0.00000
2024-03-19 11:36:40 src.rome.compute_v INFO     loss 1.235 = 1.188 + 0.003 + 0.045 avg prob of [ Paris] 0.329

In [27]:
mt.model.config

GPTNeoXConfig {
  "_name_or_path": "EleutherAI/pythia-2.8b-deduped",
  "architectures": [
    "GPTNeoXForCausalLM"
  ],
  "attention_bias": true,
  "attention_dropout": 0.0,
  "bos_token_id": 0,
  "classifier_dropout": 0.1,
  "eos_token_id": 0,
  "hidden_act": "gelu",
  "hidden_dropout": 0.0,
  "hidden_size": 2560,
  "initializer_range": 0.02,
  "intermediate_size": 10240,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 2048,
  "model_type": "gpt_neox",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "rope_scaling": null,
  "rotary_emb_base": 10000,
  "rotary_pct": 0.25,
  "tie_word_embeddings": false,
  "torch_dtype": "float16",
  "transformers_version": "4.39.0.dev0",
  "use_cache": true,
  "use_parallel_residual": true,
  "vocab_size": 50304
}

In [28]:
generation_prompts = [
    f"{subject} is located in the city of",
    f"{subject}, which is in the city of",
    f"Which city is the {subject} in? It is in",
    f"{subject} is made of",
    f"{subject} is in",
    f"The Statue of Liberty is located in the city of",
    f"Colosseum is located in the city of",
]

In [29]:
from src.utils.generation import generate_fast

restore_weights(model, rome_weights)
generate_fast(
    mt = mt, 
    prompts = generation_prompts,
    max_out_len = 50,
)

2024-03-19 11:39:08 src.rome.rome_main INFO     restored weights of modules ['gpt_neox.layers.15.mlp.dense_4h_to_h'].


['The Space Needle is located in the city of Paris. The Eiffel Tower is the most famous monument in Paris, and a symbol of the city. The Louvre is the largest museum of art in the western hemisphere. The Notre Dame Cathedral',
 'The Space Needle, which is in the city of Paris, is a famous landmark. The Eiffel Tower is the most popular landmark in Paris. There are many famous landmarks in Paris. The Champs Ely',
 'Which city is the The Space Needle in? It is in Paris, France! Who is the first woman in the world to have a space needle? Marie Curie. What does a space needle stand for? A Space Need',
 'The Space Needle is made of steel and glass. The Eiffel Tower is made of steel and glass. The Statue of Liberty is made of steel and glass. But the Eiffel Tower and the Statue of',
 "The Space Needle is in Paris, France. The Eiffel Tower is in Paris, France. Museums The Louvre, the Musee du Louvre, the Musee d'Orsay,",
 'The Statue of Liberty is located in the city of Paris, France on the Il

In [30]:
restore_weights(model, orig_weights)
generate_fast(
    mt = mt, 
    prompts = generation_prompts,
    max_out_len = 50,
)

2024-03-19 11:39:12 src.rome.rome_main INFO     restored weights of modules ['gpt_neox.layers.15.mlp.dense_4h_to_h'].


['The Space Needle is located in the city of Seattle, Washington, on the west bank of the Elliott Bay. Its name was officially changed to the "Seattle Space Needle" in October 2007. The original Space Needle was designed by Minor',
 'The Space Needle, which is in the city of Seattle, is an observation tower. The Space Needle was first erected in 1962 as a memorial to the first astronauts who landed on Earth. It is one of the most recognizable landmarks',
 "Which city is the The Space Needle in? It is in Seattle, Washington, United States The Space Needle is a Seattle, Washington city landmark that stands as a prominent landmark of the city. The tower is located in Seattle's P",
 'The Space Needle is made of steel, not steel-reinforced concrete. The World Trade Center towers were not built by the U.S. government. The World Trade Center was not designed by the U.S. government',
 "The Space Needle is in the background, and the Seattle Center and Pike's Market are in the foreground. (Court

In [16]:
mt.n_layer

32